# Single vs. Multi CRS datasets

Xproj supports both, with a user-friendly API.

In [1]:
import numpy as np
import shapely
import xarray as xr
import xvec

import xproj

xr.set_options(display_expand_indexes=True);

## Single-CRS dataset

In [2]:
random_points = shapely.points(np.random.uniform(-180, 180, 50), np.random.uniform(-90, 90, 50))

ds_single = (
    xr.Dataset(coords={"points": ("space", random_points)})
    .set_xindex("points", xvec.GeometryIndex)
    .proj.set_crs(spatial_ref="epsg:4326")
)

ds_single

<xarray.Dataset> Size: 408B
Dimensions:      (space: 50)
Coordinates:
  * points       (space) object 400B POINT (50.86996706959064 28.434822820949...
  * spatial_ref  int64 8B 0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [3]:
# user-friendly CRS access directly via the `.proj` acccessor

ds_single.proj.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
# explicit access by providing the CRS coordinate name

ds_single.proj("spatial_ref").crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Multi-CRS dataset

In [5]:
random_points2 = shapely.points(np.random.uniform(-100, 100, 20), np.random.uniform(-80, 80, 20))

ds_multi = (
    ds_single
    .assign_coords(points2=("space2", random_points2))
    .set_xindex("points2", xvec.GeometryIndex)
    .proj.set_crs(spatial_ref2="epsg:3857")
)

ds_multi

<xarray.Dataset> Size: 576B
Dimensions:       (space: 50, space2: 20)
Coordinates:
  * points        (space) object 400B POINT (50.86996706959064 28.43482282094...
  * spatial_ref   int64 8B 0
  * points2       (space2) object 160B POINT (-73.66987726534015 41.074550006...
  * spatial_ref2  int64 8B 0
Dimensions without coordinates: space, space2
Data variables:
    *empty*
Indexes:
    points        GeometryIndex (crs=None)
    spatial_ref   CRSIndex (crs=EPSG:4326)
    points2       GeometryIndex (crs=None)
    spatial_ref2  CRSIndex (crs=EPSG:3857)

In [6]:
# only explicit access by providing the CRS coordinate name

ds_multi.proj("spatial_ref").crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
ds_multi.proj("spatial_ref2").crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
# user-friendly error when trying to access directly via the `.proj` acccessor

ds_multi.proj.crs

ValueError: found multiple coordinates with a CRSIndex in Dataset or DataArray. Use instead `.proj('coord_name')` to a select a spatial reference coordinate.

In [9]:
# convenient method to check if there is one and only one CRS

ds_multi.proj.assert_single_crs()

AssertionError: multiple CRS found in Dataset or DataArray